In [1]:
import requests
import pandas as pd

### Citations:
https://eccc-msc.github.io/open-data/msc-geomet/ogc_api_en/

In [ ]:
# API endpoint
url = "https://api.weather.gc.ca/collections/climate-daily/items"

In [3]:
# Query parameters
params = {
    "datetime": "2019-01-01/2025-03-31",
    "limit": 10000,  # you can paginate if needed
}

In [4]:
# Make request
response = requests.get(url, params=params)
data = response.json()

In [5]:
# Convert to DataFrame
features = data["features"]
records = [f["properties"] for f in features]
df = pd.DataFrame(records)

In [6]:
df.size

320000

In [7]:
# Preview
display(df.head())

,PROVINCE_CODE,DAYS_WITH_PRECIP_GE_1MM,HEATING_DEGREE_DAYS,TOTAL_PRECIPITATION,DAYS_WITH_VALID_MEAN_TEMP,CLIMATE_IDENTIFIER,LATITUDE,NORMAL_PRECIPITATION,DAYS_WITH_VALID_SNOWFALL,NORMAL_SUNSHINE,...,TOTAL_SNOWFALL,FRE_PROVINCE_NAME,LOCAL_YEAR,MEAN_TEMPERATURE,MAX_TEMPERATURE,ID,MIN_TEMPERATURE,LAST_UPDATED,DAYS_WITH_VALID_PRECIP,BRIGHT_SUNSHINE
0,BC,15.0,835.4,42.8,31,1097646,52.184,45.76,31,NaN,...,39.0,COLOMBIE-BRITANNIQUE,2020,-8.948387,5.5,650.2020.1,-45.5,2020-02-03 09:45:51,31,None
1,ON,10.0,112.6,65.4,31,6100285,45.186,NaN,31,NaN,...,0.0,ONTARIO,2022,15.229032,30.5,6901.2022.5,-0.5,2022-06-03 09:47:20,31,None
2,NU,2.0,651.0,9.0,27,2301102,61.132,NaN,0,NaN,...,NaN,NUNAVUT,2020,-6.111111,5.4,7341.2020.5,-22.4,2022-06-03 09:47:26,27,None
3,ON,11.0,344.2,56.6,30,613P001,41.949,NaN,0,NaN,...,NaN,ONTARIO,2020,6.526667,19.2,27533.2020.4,-3.6,2022-05-31 14:48:04,20,None
4,ON,NaN,545.9,NaN,30,6074211,48.150,NaN,0,NaN,...,NaN,ONTARIO,2020,-0.196667,13.8,27535.2020.4,-17.9,2022-05-31 14:48:04,0,None


In [ ]:
#writeback to csv => aggregation 
df.to_csv("data/climate_data/monthly_climate_data.csv")
display(df.head())

,PROVINCE_CODE,DAYS_WITH_PRECIP_GE_1MM,HEATING_DEGREE_DAYS,TOTAL_PRECIPITATION,DAYS_WITH_VALID_MEAN_TEMP,CLIMATE_IDENTIFIER,LATITUDE,NORMAL_PRECIPITATION,DAYS_WITH_VALID_SNOWFALL,NORMAL_SUNSHINE,...,TOTAL_SNOWFALL,FRE_PROVINCE_NAME,LOCAL_YEAR,MEAN_TEMPERATURE,MAX_TEMPERATURE,ID,MIN_TEMPERATURE,LAST_UPDATED,DAYS_WITH_VALID_PRECIP,BRIGHT_SUNSHINE
0,BC,15.0,835.4,42.8,31,1097646,52.184,45.76,31,NaN,...,39.0,COLOMBIE-BRITANNIQUE,2020,-8.948387,5.5,650.2020.1,-45.5,2020-02-03 09:45:51,31,None
1,ON,10.0,112.6,65.4,31,6100285,45.186,NaN,31,NaN,...,0.0,ONTARIO,2022,15.229032,30.5,6901.2022.5,-0.5,2022-06-03 09:47:20,31,None
2,NU,2.0,651.0,9.0,27,2301102,61.132,NaN,0,NaN,...,NaN,NUNAVUT,2020,-6.111111,5.4,7341.2020.5,-22.4,2022-06-03 09:47:26,27,None
3,ON,11.0,344.2,56.6,30,613P001,41.949,NaN,0,NaN,...,NaN,ONTARIO,2020,6.526667,19.2,27533.2020.4,-3.6,2022-05-31 14:48:04,20,None
4,ON,NaN,545.9,NaN,30,6074211,48.150,NaN,0,NaN,...,NaN,ONTARIO,2020,-0.196667,13.8,27535.2020.4,-17.9,2022-05-31 14:48:04,0,None


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PROVINCE_CODE              10000 non-null  object 
 1   DAYS_WITH_PRECIP_GE_1MM    9055 non-null   float64
 2   HEATING_DEGREE_DAYS        8339 non-null   float64
 3   TOTAL_PRECIPITATION        9055 non-null   float64
 4   DAYS_WITH_VALID_MEAN_TEMP  10000 non-null  int64  
 5   CLIMATE_IDENTIFIER         10000 non-null  object 
 6   LATITUDE                   10000 non-null  object 
 7   NORMAL_PRECIPITATION       3102 non-null   float64
 8   DAYS_WITH_VALID_SNOWFALL   10000 non-null  int64  
 9   NORMAL_SUNSHINE            452 non-null    float64
 10  LOCAL_DATE                 10000 non-null  object 
 11  DAYS_WITH_VALID_MAX_TEMP   10000 non-null  int64  
 12  NORMAL_MEAN_TEMPERATURE    2838 non-null   float64
 13  NORMAL_SNOWFALL            1912 non-null   floa